In [1]:
from azureml.core import Workspace, Run
import azureml.core

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 0.1.0.1005480


In [2]:
ws = Workspace.get_or_create(name='mladsdemo', location='eastus2', subscription_id='6a275e9b-a3e6-42c2-b158-760d27f5cb83', resource_group='mladsdemo')
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: mladsdemo
Azure region: eastus2
Subscription id: 6a275e9b-a3e6-42c2-b158-760d27f5cb83
Resource group: mladsdemo


In [3]:
# get the text data from the github repo and unzip it
from fit_and_store_pipeline import unzip_file_here
import urllib
import os

if not os.path.isfile('./attack_data.csv'):
    if not os.path.isfile('./text_data.zip'): 
        urllib.request.urlretrieve('https://activelearning.blob.core.windows.net/activelearningdemo/text_data.zip', 'text_data.zip')
    unzip_file_here('text_data.zip')

if not os.path.isfile('miniglove_6B_50d_w2v.txt'):
    unzip_file_here('miniglove_6B_50d_w2v.zip')
    
print('Data files here')

C:\Users\tosingli\AppData\Local\conda\conda\envs\myenv\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Data files here


In [4]:
from fit_and_store_pipeline import *
script_main()

[[836  14]
 [ 86  64]]
{"result": [1, 0, 0, 0, 1]}


In [110]:
%%writefile myenv.yml

# my attempt 1 at env file: packages are in pip
name: myenv
channels:
  - defaults
dependencies:
  - pip:        
    - scikit-learn
    - numpy
    - scipy
    - numexpr
    - gensim
    # Required packages for AzureML execution, history, and data preparation.
    - --extra-index-url https://azuremlsdktestpypi.azureedge.net/sdk-release/Preview/E7501C02541B433786111FE8E140CAA1
    - azureml-core

Overwriting myenv.yml


In [5]:
%%writefile myenv.yml

# give this a try: packages are in conda
name: myenv
channels:
  - defaults
dependencies:
  - scikit-learn
  - pandas
  - gensim
  - numpy
  - pip:
    # Required packages for AzureML execution, history, and data preparation.
    - --extra-index-url https://azuremlsdktestpypi.azureedge.net/sdk-release/Preview/E7501C02541B433786111FE8E140CAA1
    - azureml-core

Overwriting myenv.yml


In [6]:
# you may need to register the provider in your subscription 
# az provider register -n Microsoft.ContainerInstance

from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 7, 
                                               tags = ['MLADS 2018 Text Workshop'], 
                                               description = 'Sample container deployment')

In [7]:
del service

NameError: name 'service' is not defined

In [ ]:
%%time
# this will take 5-10 minutes to finish
service = ws.deploy_webservice(name = 'aci-attack-clsfr1',
                               deploy_config = aciconfig,
                               target = None,
                               models = ['rf_attack_classifier_pipeline.pkl'],
                               runtime = 'python',
                               conda_file = 'myenv.yml',
                               driver = 'fit_and_store_pipeline.py')
service.wait_for_deployment(show_output = True)

Registering model rf_attack_classifier_pipeline.


In [72]:
# Let's try to do it the "4 steps" way
# Step 1: Make an image, this usually succeeds
# image = ws.create_image(name, models, runtime, driver_file, docker_file=None, conda_file=None, dependencies=None, schema_file=None, enable_gpu=False, tags=None, description=None)
img = ws.create_image(name = 'aci-attack-clsfr4-img',                               
                      models = ['rf_attack_classifier_pipeline.pkl'],
                      runtime = 'python',
                      conda_file = 'myenv.yml',
                      driver_file = 'fit_and_store_pipeline.py')

KeyboardInterrupt: 

In [74]:
# We can also get the image by name from the Workspace
img = ws.get_image(id="aci-attack-clsfr4:1")

In [82]:
# Let's try to do it the "4 steps" way
# Step 2: Deploy image, this usually times out
from azureml.core.webservice import Webservice

service = Webservice.deploy_from_image(workspace=ws,
                                       name = 'aci-attack-clsfr5-srv',
                                       image=img,
                                       deployment_config = aciconfig)

Creating service


In [85]:
# we can get the service by name
service = ws.get_webservice(name='aci-attack-clsfr4-srv')

In [86]:
# verify the deployed service
print('web service is hosted in ACI:', service.scoring_uri)

web service is hosted in ACI: http://138.91.127.140:5001/score


In [9]:
# get service after timeout
from azureml.core.webservice import Webservice
services = Webservice.list(workspace=ws)
print(services)
print(len(services))
# l1 = services[0].get_logs()
# l2 = services[1].get_logs()
# l1
service = services[2]

1


IndexError: list index out of range

In [10]:
# delete existing services before deploying new ones
services = Webservice.list(workspace=ws)
for i in range(0, len(services)):
    try:
        services[i].delete()
    except:
        pass

In [11]:
# delete failed images
from azureml.core.image import Image
images = Image.list(workspace=ws)
for i in range(0, len(images)):
    try:
        images[i].delete()
    except:
        pass

In [89]:
# Now score the webservice

import json

test_attacks = [ 'You are scum.', 'I like your shoes.', 'You are pxzx.', 
                 'Your mother was a hamster and your father smelt of elderberries',
                 'Buy some hair slime for me, please']

# score the first row from the test set.
test_samples = bytes(json.dumps({"data": test_attacks}), encoding = 'utf8')
test_samples

b'{"data": ["You are scum.", "I like your shoes.", "You are pxzx.", "Your mother was a hamster and your father smelt of elderberries", "Buy some hair slime for me, please"]}'

In [90]:
service.run(input_data = test_samples)

KeyboardInterrupt: 

In [123]:
service.get_logs()

'2018-06-11 23:39:44,583 CRIT Supervisor running as root (no user in config file)\n2018-06-11 23:39:44,585 INFO supervisord started with pid 1\n2018-06-11 23:39:45,587 INFO spawned: \'rsyslog\' with pid 8\n2018-06-11 23:39:45,589 INFO spawned: \'program_exit\' with pid 9\n2018-06-11 23:39:45,590 INFO spawned: \'nginx\' with pid 10\n2018-06-11 23:39:45,592 INFO spawned: \'iot\' with pid 11\n2018-06-11 23:39:45,593 INFO spawned: \'gunicorn\' with pid 12\n2018-06-11 23:39:45,682 INFO success: iot entered RUNNING state, process has stayed up for > than 0 seconds (startsecs)\nEdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...\n2018-06-11 23:39:45,756 INFO exited: iot (exit status 1; expected)\n2018-06-11 23:39:46,757 INFO success: rsyslog entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)\n2018-06-11 23:39:46,758 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)\n2018-06-11 23:39:50,762